# Yielding and Generators

### Review

In [15]:
import math

class FactIter:
    def __init__( self, n ):
        self.n = n
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = math.factorial(self.i)
            self.i += 1
            return result
        

In [3]:

# What if we could do something like this instead:

def factorials(n):
    for i in range(n):
        # emit factorial(i)
        # Pause execution here
        # Wait for resume
        return 'done!'


And in our code we would want to do something like this maybe:<br>

facts = factorials(4)<br>

get_next(facts) → 0!<br>
get_next(facts) → 1!<br>
get_next(facts) → 2!<br>
get_next(facts) → done!<br>

Of course, getting 0!, 1!, 2!, 3! followed by a string is odd<br>
And what happens if we call `get_next` again?<br>

Maybe we should consider raising an exception… `StopIteration` ?<br>
And instead of calling `get_next`, why not just use `next` ?<br>
But what about that `emit`, `pause`, `resume` ? → `yield`<br>


The `yield` keyword does exactly what we want:<br>
it `emits` a value<br>
the function is effectively `suspended` (but it retains its current state)<br>
calling `next` on the function `resumes` running the function right after the yield statement<br>
if function `returns` something instead of yielding (finishes running) → `StopIteration` exception<br>
    

In [4]:

def song():
    print("line 1")
    yield "Im ok"
    print('line 2')
    yield "I work"
    

In [6]:
lines = song() # -> no output

line = next(lines) # -> current control in line 'Im Ok'

line 1


In [7]:
line = next(lines) # current control in line 'I work'

line 2


In [8]:
line = next(lines) # StopIteration

StopIteration: 

## Generators<br>

A function that uses the `yield` statement, is called a generator function<br>


In [6]:

def my_func():
    yield 1
    yield 2
    yield 3
    
    

In [7]:
gen = my_func() # gen object creates

In [8]:
next( gen ) 

1

function body will execute until it encounters a `yield` statement ( return value of next() ),then it, `suspends` itself.<br>
until `next` is called again → suspended function `resumes` execution<br>

if it encounters a return before a `yield`<br>
→ `StopIteration` exception occurs

In [9]:
next( gen ) 
next( gen ) 

3

In [10]:
next( gen ) 

StopIteration: 

### Generators are iterators.<br>

They implement the **ierator protocol**. `__iter__` and `__next__`<br>

they are `exhausted` when function `return` a value<br>
-> `StopIteration` exception.<br>
-> return value is the exception `message`.<br>

In [11]:
gen = my_func()

In [13]:
gen.__iter__() # iter(gen)

<generator object my_func at 0x00000233127E6AC0>

In [14]:
gen.__next__() # next(gen)

1

In [17]:
fact_iter = FactIter(5)

list( fact_iter )

[1, 1, 2, 6, 24]

In [19]:
# similary using generators

def factorials(n):
    for i in range(n):
        yield math.factorial(i)
        
f = factorials(5)

list( f )

[1, 1, 2, 6, 24]

Generator functions are functions which contain at least one `yield` statement.<br>
When a generator function is called, Python `creates` a `generator` object<br>

Generators implement the `iterator` protocol<br>

Generators are inherently lazy iterators ( can be infinite )<br>

Generators are iterators, and can be used in the same way (for loops, comprehensions, etc)<br>

Generators become `exhausted` once the function returns a value or None.<br>



## Generator Expressions<br>

Generator expressions use the same comprehension syntax but instead of using `[]` and we use `()`.<br>


List Comprehensions

In [22]:
l = [i ** 2 for i in range(5)] # returns a list

l

[0, 1, 4, 9, 16]

In [21]:

[
    (i,j)
    for i in range(1,6) if i % 2 == 0
    for j in range(1,6) if j % 3 == 0
]

[(2, 3), (4, 3)]

Difference

In [24]:
g = (i ** 2 for i in range(5)) #a generator is returned

List comprehensions are `eager`<br>
all objects are created right away<br>
→ takes `longer` to `create/return` the list<br>
→ `iteration` is `faster` (objects already created)<br>
→ entire collections is loaded into memory

Generators are `lazy`<br>
object creation is delayed until requested by `next()`<br>
→ generator is `created`/returned `immediately`<br>
→ iteration is `slower` (objects need to be created)
→ only a `single` item is loaded at a time
in general, generators tend to have `less` memory overhead

In [25]:
import dis

In [26]:
exp = compile('[i**2 for i in range(5)]', filename='<string>', mode='eval')

In [27]:
dis.dis(exp)

  1           0 LOAD_CONST               0 (<code object <listcomp> at 0x0000023312835190, file "<string>", line 1>)
              2 LOAD_CONST               1 ('<listcomp>')
              4 MAKE_FUNCTION            0
              6 LOAD_NAME                0 (range)
              8 LOAD_CONST               2 (5)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 RETURN_VALUE

Disassembly of <code object <listcomp> at 0x0000023312835190, file "<string>", line 1>:
  1           0 BUILD_LIST               0
              2 LOAD_FAST                0 (.0)
        >>    4 FOR_ITER                12 (to 18)
              6 STORE_FAST               1 (i)
              8 LOAD_FAST                1 (i)
             10 LOAD_CONST               0 (2)
             12 BINARY_POWER
             14 LIST_APPEND              2
             16 JUMP_ABSOLUTE            4
        >>   18 RETURN_VALUE


In [28]:
exp = compile('(i ** 2 for i in range(5))', filename='<string>', mode='eval')

In [29]:
dis.dis(exp)

  1           0 LOAD_CONST               0 (<code object <genexpr> at 0x0000023312835710, file "<string>", line 1>)
              2 LOAD_CONST               1 ('<genexpr>')
              4 MAKE_FUNCTION            0
              6 LOAD_NAME                0 (range)
              8 LOAD_CONST               2 (5)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 RETURN_VALUE

Disassembly of <code object <genexpr> at 0x0000023312835710, file "<string>", line 1>:
  1           0 LOAD_FAST                0 (.0)
        >>    2 FOR_ITER                14 (to 18)
              4 STORE_FAST               1 (i)
              6 LOAD_FAST                1 (i)
              8 LOAD_CONST               0 (2)
             10 BINARY_POWER
             12 YIELD_VALUE
             14 POP_TOP
             16 JUMP_ABSOLUTE            2
        >>   18 LOAD_CONST               1 (None)
             20 RETURN_VALUE


Observation: Internal mechanism is same. only difference is on how a function is created?<br>
List is created an `iterable`.<br>
While other a generator creates an `iterator`.

#### Memory Usage.<br>
Example, HP-UX story



# yield from

In [34]:

def matrix(n):
    gen = ( ( i * j for j in range(1, n + 1 ))
          for i in range(1, n+1 ) 
          )
    return gen

In [35]:
m = list(matrix(5))
m

[<generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000023312ACEF20>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000023312ACE9E0>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000023312ACEBA0>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000023312ACE660>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x0000023312ACE430>]

In [36]:
def matrix_iterator(n):
    for row in matrix(n):
        for item in row:
            yield item
            

In [37]:
for i in matrix_iterator(3):
    print(i)

1
2
3
2
4
6
3
6
9


In [38]:
def matrix_iterator(n):
    for row in matrix(n):
        yield from row

In [39]:
for i in matrix_iterator(3):
    print(i)

1
2
3
2
4
6
3
6
9


We can think of 
```
yield from <iterator>
```
as a replacement for the code:
```
for i in <iterator>:
    yield i
```

In [40]:
brands = []

with open('car-brands-1.txt') as f:
    for brand in f:
        brands.append(brand.strip('\n'))
        
with open('car-brands-2.txt') as f:
    for brand in f:
        brands.append(brand.strip('\n'))
        
with open('car-brands-3.txt') as f:
    for brand in f:
        brands.append(brand.strip('\n'))
        

In [41]:
for brand in brands:
    print(brand, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citroën, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Å koda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 

In [42]:
def brands(*files):
    for f_name in files:
        with open(f_name) as f:
            for line in f:
                yield line.strip('\n')

In [43]:
files = 'car-brands-1.txt', 'car-brands-2.txt', 'car-brands-3.txt'
for brand in brands(*files):
    print(brand, end = ', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citroën, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Å koda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 

We can simplify our function by using `yield from`:

In [44]:
def brands(*files):
    for f_name in files:
        with open(f_name) as f:
            yield from f

In [45]:
for brand in brands(*files):
    print(brand, end=', ')

Alfa Romeo
, Aston Martin
, Audi
, Bentley
, Benz
, BMW
, Bugatti
, Cadillac
, Chevrolet
, Chrysler
, Citroën
, Corvette
, DAF
, Dacia
, Daewoo
, Daihatsu
, Datsun
, De Lorean
, Dino
, Dodge, Farboud
, Ferrari
, Fiat
, Ford
, Honda
, Hummer
, Hyundai
, Jaguar
, Jeep
, KIA
, Koenigsegg
, Lada
, Lamborghini
, Lancia
, Land Rover
, Lexus
, Ligier
, Lincoln
, Lotus
, Martini, Maserati
, Maybach
, Mazda
, McLaren
, Mercedes-Benz
, Mini
, Mitsubishi
, Nissan
, Noble
, Opel
, Peugeot
, Pontiac
, Porsche
, Renault
, Rolls-Royce
, Saab
, Seat
, Å koda
, Smart
, Spyker
, Subaru
, Suzuki
, Toyota
, Vauxhall
, Volkswagen
, Volvo, 

In [46]:
def gen_clean_read(file):
    with open(file) as f:
        for line in f:
            yield line.strip('\n')

In [47]:
f1 = gen_clean_read('car-brands-1.txt')
for line in f1:
    print(line, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citroën, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, 

In [48]:
files = 'car-brands-1.txt', 'car-brands-2.txt', 'car-brands-3.txt'

In [49]:
def brands(*files):
    for file in files:
        yield from gen_clean_read(file)

In [50]:
for brand in brands(*files):
    print(brand, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citroën, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Å koda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 

In [51]:
def brands(*files):
    for file in files:
        yield from gen_clean_read(file)

In [52]:
def brands(*files):
    for file in files:
        for line in gen_clean_read(file):
            yield line

In [53]:
for brand in brands(*files):
    print(brand, end=', ')

Alfa Romeo, Aston Martin, Audi, Bentley, Benz, BMW, Bugatti, Cadillac, Chevrolet, Chrysler, Citroën, Corvette, DAF, Dacia, Daewoo, Daihatsu, Datsun, De Lorean, Dino, Dodge, Farboud, Ferrari, Fiat, Ford, Honda, Hummer, Hyundai, Jaguar, Jeep, KIA, Koenigsegg, Lada, Lamborghini, Lancia, Land Rover, Lexus, Ligier, Lincoln, Lotus, Martini, Maserati, Maybach, Mazda, McLaren, Mercedes-Benz, Mini, Mitsubishi, Nissan, Noble, Opel, Peugeot, Pontiac, Porsche, Renault, Rolls-Royce, Saab, Seat, Å koda, Smart, Spyker, Subaru, Suzuki, Toyota, Vauxhall, Volkswagen, Volvo, 